# Simulating with Census Data

# Step 1 - process your census data
On Your Own - here is an example

In [3]:
## All data files for step 2

# base_population
base_pop_data_path = ''
base_pop_mapping = ''
base_pop_out_file = 'base_population.pkl'

# household
household_data_path = ''
household_mapping = ''
household_out_file = 'household.pkl'

In [4]:
## Executing step 2

from AgentTorch.loaders import CensusDataGenerator
census_loader = CensusDataGenerator()

census_loader.generate_basepop(base_pop_data_path, base_pop_mapping, base_pop_out_file)
census_loader.generate_households(household_data_path, household_mapping, household_out_file)

population_df = census_loader.population_df # this can be used directly for simulation

In [ ]:
## Step 3 - Run an existing simulation
from AgentTorch.simulation import load_sim, run_sim
from AgentTorch.models import covid

covid_config = load_sim(covid)
run_sim(covid_config, population_df)

In [ ]:
'''Existing simulation exists - rerun in few lines of code'''
def run_sim(config, population_df):
    assert column_set
    
    config_data_path = preprocess(population_df)
    config.set_data(config_data_path)
    
    runner = Runner(config)
    runner.execute()

In [7]:
import pandas as pd
import torch


def load_population(df_path,attribute):
    """
    Load population data from a pandas dataframe
    """
    # Load population data
    df = pd.read_pickle(df_path)
    df = df['synpop'][attribute]
    df,mapping = pd.factorize(df)
    att_tensor = torch.from_numpy(df).float()
    return att_tensor,mapping

In [5]:
df_path = '/tmp/syspop_test/NYC/1/tmp/synpop.pickle'

In [ ]:
pop = load_population(df_path,'area')


In [ ]:
pop

In [ ]:
# pop = pop.values
pop = torch.from_numpy(pop).float()
pop

In [ ]:
df = pop['synpop']['area']
df = pd.get_dummies(df, columns=['area']).astype(int)
df

In [13]:
df = pd.read_pickle(df_path)
gender = df['synpop']['gender']
gender,map = pd.factorize(gender)


In [15]:
mapping_collection = {}
mapping_collection['gender'] = map
mapping_collection

dict_values([Index(['female', 'male'], dtype='object')])

In [16]:
df = pd.read_pickle(df_path)
age = df['synpop']['age']
age,map = pd.factorize(age)

In [17]:
mapping_collection['age'] = map

In [18]:
mapping_collection

{'gender': Index(['female', 'male'], dtype='object'),
 'age': Index(['U19', '20t29', '30t39', '40t49', '50t64', '65A'], dtype='object')}

In [10]:
df.keys()

dict_keys(['synpop', 'synadd'])

In [9]:
df['synpop'].keys()

Index(['area', 'age', 'gender', 'ethnicity'], dtype='object')

In [ ]:
f

In [ ]:
df['synpop']['area']

In [19]:
df = pd.read_pickle("/tmp/syspop_test/NYC/1/tmp/area.pickle")

In [22]:
att_tensor = torch.from_numpy(df.values).float()
att_tensor

tensor([ 0.,  0.,  0.,  ..., 32., 32., 32.])

# Prompt

In [ ]:
####
import os
####
import torch
import random
import numpy as np

def set_seeds(seed):    
    random.seed(seed)
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seeds(42)
####
from accelerate import Accelerator

from accelerate.utils import gather_object
from transformers import AutoModelForCausalLM, AutoTokenizer
from statistics import mean
import torch, time, json

accelerator = Accelerator()
####
import json
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import CLIPProcessor, CLIPModel


In [ ]:
prompt_template = "You are {male} of age {age} and are {ethnicity}. You live in {area}. Give your willingness to work in {industry}, denote the willingness by giving a value between 0 and 1, with 0 being not willing at all and 1 being completely willing."

In [ ]:
get_agent_age = "U19"
get_agent_gender = "Male"
get_agent_ethnicity = "White"
area = "Manhattan"
industry = "Finance"
prompt = prompt_template.format(male = get_agent_gender,age = get_agent_age,ethnicity=get_agent_ethnicity,industry=industry,area=area)

In [ ]:
prompt

In [ ]:
torch_dtype = torch.bfloat16
model_id = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id)

model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    device_map={"": accelerator.device},
    torch_dtype=torch_dtype, 
    low_cpu_mem_usage=True,
)
    
model.eval()